<a href="https://colab.research.google.com/github/37stu37/FFE/blob/master/temporal_network_for_FFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install memory_profiler

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Imports**
---



In [0]:
import numpy as np
import pandas as pd
from pathlib import Path
import memory_profiler as mem_profile
import sys
import os
import glob
import multiprocessing
# import zipfile
from zipfile import ZipFile 


pd.options.mode.chained_assignment = None  # default='warn'

**Load data from zip file**
---



In [0]:
# import data from zip file
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/runs')
with ZipFile(folder/'data.zip', 'r') as zip: 
    edge_file = zip.extract('Copy of edge_data.parquet')
    wind_file = zip.extract('Copy of GD_wind.csv') 

In [0]:
# load data
wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

**Definitions**
---



In [0]:
# %%timeit
def wind_scenario(wind_data):
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(edges=edgelist):
    rng = np.random.uniform(0, 1, size=edges.values.shape[0])
    mask = rng < edges.IgnProb_bl.values
    NewActiveEdges = edges[mask]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


In [0]:
def clean_up(path):
    files = glob.glob(path)
    print(" {} files removed".format(len(files)))
    for f in files:
      os.remove(f)

**Main**
---


---



In [0]:
def ffe_runs(n):
    listScenarioDataframes = []
    for scenario in range(n):
        # initial setup
        condition = True
        # listScenarioDataframes = []
        listActivatedSources = []
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges = ignition()
        if ActiveEdges.empty:
            continue
        while condition:
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty:
                break
            listScenarioDataframes.append(ActiveEdges)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print("scenario : {}, process id: {}".format(scenario, os.getpid()))
        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["pid"] = os.getpid()
        Activations.to_parquet(folder / 'output' / 'scenario{}_pid{}_Activations.parquet'.format(scenario, os.getpid()), engine='pyarrow')

In [9]:
# run process on all available cores - need to input number of scenarios required
numberOfScenariosPerCore = 1500
%%time
if __name__ == '__main__':
    # what are your inputs, and what operation do you want to
    # perform on each input. For example...
    inputs = range(numberOfScenariosPerCore) # range is the number of scenarios
    #  removing processes argument makes the code run on all available cores
    pool = mp.Pool()
    results = pool.map(ffe_runs, inputs)

SyntaxError: ignored


**Backup**
---



---



In [0]:
# clean_up('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/runs/output/*')

In [0]:
# pqt = pd.read_parquet("/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/runs/output/scenario0_pid1998_Activations.parquet")

In [0]:
# num_cores = multiprocessing.cpu_count()
# print(num_cores)